In [1]:
import chess

In [2]:
board = chess.Board()

In [3]:
import time
from IPython.display import display, HTML, clear_output

In [4]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

In [5]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

In [6]:
def play_game(player1, player2, visual="svg", pause=0.1):
    """
    playerN1, player2: functions that takes board, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board()
    try:
        planilla = []
        numero_jugada = 1.0
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
            planilla.append(str(numero_jugada) + '. ' + uci)
            print(planilla)
            numero_jugada = numero_jugada + 0.5 
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    return (result, msg, board)

In [7]:
import random

In [8]:
def random_player(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

In [9]:
def human_player(board):
    display(board)
    uci = get_move("%s's move [q to quit]> " % who(board.turn))
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print("Legal moves: " + (",".join(sorted(legal_uci_moves))))
        uci = get_move("%s's move[q to quit]> " % who(board.turn))
    return uci

In [10]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

In [11]:
def player1(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis1(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [12]:
def staticAnalysis1(board, move, my_color):
    score = 0
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    return score

In [13]:
def player2(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis2(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [14]:
def staticAnalysis2(board, move, my_color):                              # mejora de la funcion de analisis anterior
    score = random.random()
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    return score

In [15]:
def player3(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis3(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [16]:
def staticAnalysis3(board, move, my_color):                             # mejora de la funcion de analisis anterior
    score = random.random()
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    score += 100 if board.is_checkmate() else 0
    return score

In [17]:
def player4(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        score = random.random()
        move.score = minimax(newboard, move, board.turn, score, depth = 0)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [18]:
def minimax(board, move, my_color, score, depth):
    if depth == 0:
        return staticAnalysis4(board, move, my_color, score)
    else:
        if chess.WHITE:
            bestscore = -float("inf")
            board.push(move)
            for another_move in list(board.legal_moves):
                newboard = board.copy()
                score = minimax(newboard, move, board.turn, score, depth - 1)
                if score > bestscore:
                    bestscore = score
            return bestscore
        else:
            bestscore = float("inf")
            board.push(move)
            for another_move in list(board.legal_moves):
                newboard = board.copy()
                score = minimax(newboard, move, board.turn, score, depth - 1)
                if score < bestscore:
                    bestscore = score
            return bestscore

In [19]:
def staticAnalysis4(board, move, my_color, score):                             # mejora de la funcion de analisis anterior
    ## Check some things about this move:
    score += 1 if board.is_capture(move) else 0
    score += 1 if board.is_castling(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 3), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 9), 
                           (chess.KNIGHT, 3),
                           (chess.ROOK, 5)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    score += 100 if board.is_checkmate() else 0
    score -= 100 if board.is_stalemate() else 0
    score -= 0 if board.has_castling_rights(my_color) else 1
    score += 0 if board.has_castling_rights(not my_color) else 1
    return score

In [20]:
first_game = play_game(player4, player4)

['1.0. b1a3', '1.5. a7a5', '2.0. h2h3', '2.5. e7e5', '3.0. g2g4', '3.5. f8a3', '4.0. b2a3', '4.5. f7f5', '5.0. g4f5', '5.5. d7d5', '6.0. f5f6', '6.5. g8f6', '7.0. c1b2', '7.5. c8h3', '8.0. g1h3', '8.5. d8d7', '9.0. b2e5', '9.5. d7h3', '10.0. h1h3', '10.5. f6g8', '11.0. h3h7', '11.5. h8h7', '12.0. e5g7', '12.5. h7g7', '13.0. d1b1', '13.5. g7g1', '14.0. b1b7', '14.5. g1f1', '15.0. e1f1', '15.5. b8a6', '16.0. b7a8', '16.5. e8e7', '17.0. a8a6', '17.5. e7e8', '18.0. a6a5', '18.5. c7c6', '19.0. a5d5', '19.5. c6d5', '20.0. d2d3', '20.5. e8e7', '21.0. a1b1', '21.5. d5d4', '22.0. b1e1', '22.5. e7f7', '23.0. f1g1', '23.5. f7g7', '24.0. f2f3', '24.5. g7g6', '25.0. c2c4', '25.5. d4c3', '26.0. g1f2', '26.5. g6f7', '27.0. e1h1', '27.5. f7f6', '28.0. h1a1', '28.5. c3c2', '29.0. a1d1', '29.5. c2d1q', '30.0. f2g3', '30.5. d1e2', '31.0. a3a4', '31.5. e2d3', '32.0. g3g2', '32.5. d3f3', '33.0. g2f3', '33.5. g8h6', '34.0. f3f2', '34.5. f6e6', '35.0. f2f3', '35.5. e6e5', '36.0. a2a3', '36.5. e5e6', '37.0. f

In [21]:
3+5

8